In [1]:
from selenium.webdriver.common.by import By
from datetime import datetime
import selenium as sel
from tqdm import tqdm
import pandas as pd
import base as bs
import os

In [2]:
#Empresa con la cual vamos a extraer los articulos
empresa = input("Digite la empresa a extraer: ").lower().strip()
if " " in empresa:
    empresa_ = empresa.replace(" ", "%20")
else:
    empresa_ = empresa
revista = "elColombiano"

In [17]:
# crear driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
try:
    driver = sel.webdriver.Edge()
except:
    cwd = os.getcwd()
    path = os.path.join(cwd, 'msedgedriver.exe')
    path.replace("\\\\", "\\")
    driver = sel.webdriver.Edge(executable_path=path.replace("\\\\", "\\"))
driver.get(f'https://www.elcolombiano.com/busqueda/-/search/{empresa_}/false/false/19810311/20230311/date/true/true/0/0/meta/0/0/0/1')
driver.maximize_window()

In [4]:
# Articulos a extraer
num_paginas = input("Ingrese el numero de páginas que desea recorrer").lower()
url_princ = driver.current_url[:-1]
titulares = []
for i in tqdm(range(1, int(num_paginas) + 1)):
    aux = str(i)
    url_a_buscar = url_princ+aux
    driver.get(url_a_buscar)
    articulos = driver.find_elements(By.XPATH, './/li[@class="element   full-access norestricted"]')
    
    for art in tqdm(articulos):
        url = art.find_element(By.XPATH, './/div[contains(@class, "right")]//a').get_attribute('href')
        if not(bs.existedb(url, "elColombiano")):
            titulo = art.find_element(By.XPATH, './/h3[contains(@class, "titulo-noticia")]//span').text
            fechaP = art.find_element(By.XPATH, './/div[contains(@class, "fecha")]//span').text
            fechaP = datetime.strptime(fechaP, '%d / %m / %Y')
            tema = art.find_element(By.XPATH, './/div[contains(@class, "information")]//a').text
            autor = art.find_element(By.XPATH, './/span[contains(@class, "autor")]').text
            lista_tags = art.find_element(By.XPATH, './/div[contains(@class, "tags-noticia")]//ul')
            tags = list(map(lambda x: x.text, lista_tags.find_elements(By.XPATH, './/li')))
            imagen = bs.obtener_imagen_col(art)
            titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                            'Titulo': titulo,
                            'Fecha Publicacion': fechaP,
                            'URL': url,
                            'Tema': tema,
                            'Autor': autor,
                            'Tags': tags,
                            'Imagen': imagen,
                            'Empresa': empresa,
                            'Fuente': 'El Colombiano'
                            })

100%|██████████| 3/3 [00:25<00:00,  8.36s/it]


In [5]:
for tit in tqdm(titulares):
    driver.get(tit["URL"])
    driver.implicitly_wait(10)
    tit["Contenido"] = bs.obtener_contenido_col(driver)
    tit["Resumen"] = bs.obtener_resumen_col(driver)

100%|██████████| 15/15 [04:23<00:00, 17.59s/it]


In [6]:
df = pd.DataFrame(titulares)
# bs.writeData("database", df)

In [8]:
driver.quit()